<a href="https://colab.research.google.com/github/amimulhasan/Deep-Learning/blob/main/new_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import zipfile

zip_path = '/content/drive/MyDrive/MRI_DATA_IMAGE.zip'
extract_to = 'brain_tumor_data'

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to)

print("Unzipping completed!")

Unzipping completed!


In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_hub as hub
import numpy as np
import matplotlib.pyplot as plt


In [ ]:
img_size = 224
batch_size = 32
data_dir = "/content/brain_tumor_data/MRI_DATA_IMAGE"  # This contains subfolders for each class

# Load training dataset (80% of data)
train_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,      # reserve 20% for validation
    subset="training",
    seed=123,                  # for reproducibility
    image_size=(img_size, img_size),
    batch_size=batch_size,
    label_mode='int'
)

# Load validation dataset (20% of data)
val_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=(img_size, img_size),
    batch_size=batch_size,
    label_mode='int'
)

class_names = train_ds.class_names
num_classes = len(class_names)

print("Classes:", class_names)


Found 1176 files belonging to 2 classes.
Using 941 files for training.
Found 1176 files belonging to 2 classes.
Using 235 files for validation.
Classes: ['Normal', 'Tumor Patient']


In [ ]:
normalization_layer = layers.Rescaling(1./255)

train_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
val_ds = val_ds.map(lambda x, y: (normalization_layer(x), y))


In [ ]:
def build_cnn_branch(input_shape):
    inputs = keras.Input(shape=input_shape)
    x = layers.Conv2D(32, 3, activation='relu')(inputs)
    x = layers.MaxPooling2D()(x)
    x = layers.Conv2D(64, 3, activation='relu')(x)
    x = layers.MaxPooling2D()(x)
    x = layers.Conv2D(128, 3, activation='relu')(x)
    x = layers.GlobalAveragePooling2D()(x)
    return keras.Model(inputs, x, name="cnn_branch")



In [ ]:
def build_vit_branch(input_shape):
    vit_url = "https://tfhub.dev/google/vit/base_patch16_224/1"
    vit_layer = hub.KerasLayer(vit_url, trainable=False)

    inputs = keras.Input(shape=input_shape)
    x = vit_layer(inputs)  # output shape: [batch, 768]
    return keras.Model(inputs, x, name="vit_branch")


In [ ]:
def build_lstm_branch(input_shape):
    inputs = keras.Input(shape=input_shape)
    # Reshape to sequence length = width*height, features = channels
    x = layers.Reshape((input_shape[0]*input_shape[1], input_shape[2]))(inputs)
    x = layers.LSTM(64)(x)
    return keras.Model(inputs, x, name="lstm_branch")


In [ ]:
input_shape = (img_size, img_size, 3)

cnn_branch = build_cnn_branch(input_shape)
vit_branch = build_vit_branch(input_shape)
lstm_branch = build_lstm_branch(input_shape)

inputs = keras.Input(shape=input_shape)
cnn_out = cnn_branch(inputs)
vit_out = vit_branch(inputs)
lstm_out = lstm_branch(inputs)

concat = layers.concatenate([cnn_out, vit_out, lstm_out])
x = layers.Dense(256, activation='relu')(concat)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(num_classes, activation='softmax')(x)

model = keras.Model(inputs, outputs)


OSError: https://tfhub.dev/google/vit/base_patch16_224/1 does not appear to be a valid module.